In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from sklearn.decomposition import PCA
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import DataLoader, random_split
os.makedirs("images", exist_ok=True)



## Génération des images

In [4]:
def generate_image(a, b, c, idx):
    x = np.linspace(-10, 10, 100)
    y = a * x**2 + b * x + c
    plt.figure(figsize=(2, 2), dpi=32)
    plt.plot(x, y, color='black', linewidth=3)
    plt.ylim(-200, 200)
    plt.axis('off')
    plt.tight_layout()
    path = f"images/img_{idx:03}.png"
    plt.savefig(path, bbox_inches='tight', pad_inches=0)
    plt.close()
    return path

## Génération du fichier paramètres

In [7]:
def generate_excel(param_list, path='params.xlsx'):
    df = pd.DataFrame(param_list, columns=['a', 'b', 'c'])
    df.to_excel(path, index=False)


In [9]:
def generate_all_images_from_excel(excel_path='params.xlsx'):
    df = pd.read_excel(excel_path)
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        generate_image(row['a'], row['b'], row['c'], idx)

## Construction du MLP

In [12]:
class ImageRegressionDataset(Dataset):
    def __init__(self, excel_path, image_folder):
        self.df = pd.read_excel(excel_path)
        self.image_folder = image_folder
        self.transform = transforms.Compose([
            transforms.Resize((128, 128)),
            transforms.ToTensor(),  
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        params = self.df.iloc[idx][['a', 'b', 'c']].values.astype(np.float32)
        img_path = os.path.join(self.image_folder, f"img_{idx:03}.png")
        image = Image.open(img_path).convert('L')  
        image = self.transform(image)
        return torch.tensor(params), image


In [14]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(3, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 128*128),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)


In [16]:
def main_training():
    # Chargement du dataset complet
    dataset = ImageRegressionDataset('params.xlsx', 'images')

    # Définition des tailles pour train / val / test
    train_size = int(0.7 * len(dataset))
    val_size = int(0.15 * len(dataset))
    test_size = len(dataset) - train_size - val_size

    # Découpage aléatoire
    train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

    # Modèle, optimizer et loss
    model = MLP()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.MSELoss()

    # Boucle d'entraînement
    for epoch in range(50):
        model.train()
        train_loss = 0
        for inputs, targets in train_loader:
            targets = targets.view(inputs.size(0), -1)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for inputs, targets in val_loader:
                targets = targets.view(inputs.size(0), -1)
                outputs = model(inputs)
                val_loss += criterion(outputs, targets).item()
        val_loss /= len(val_loader)

        print(f"Epoch {epoch+1}, Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")

    # Sauvegarde du modèle pour plus tard
    torch.save(model.state_dict(), 'mlp_weights.pth')

## Inference

In [19]:
def main_inference(a, b, c, output_folder='predictions'):
    os.makedirs(output_folder, exist_ok=True)

    model = MLP()
    model.load_state_dict(torch.load('mlp_weights.pth'))
    model.eval()

    with torch.no_grad():
        input_tensor = torch.tensor([[a, b, c]], dtype=torch.float32)
        output = model(input_tensor).view(128, 128).numpy()
        output[output < 0.8] *= 0.2 
        plt.imshow(output, cmap='gray')
        plt.axis('off')
        image_path = os.path.join(output_folder, f"prediction_a{a}_b{b}_c{c}.png")
        plt.savefig(image_path, bbox_inches='tight', pad_inches=0)
        plt.close()

    print("Image sauvegardée dans le fichier predictions")

## Entraînement du modèle

In [56]:
a_vals = np.linspace(1, 3, 10)
b_vals = np.linspace(10, 30, 10)
c_vals = np.linspace(1, 3, 10)
param_list = [(a, b, c) for a in a_vals for b in b_vals for c in c_vals]
generate_excel(param_list)
generate_all_images_from_excel()

# Entraînement
main_training()

# Reconstruction du test set
dataset = ImageRegressionDataset('params.xlsx', 'images')
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
_, _, test_dataset = random_split(dataset, [train_size, val_size, test_size])
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Chargement du modèle
model = MLP()
model.load_state_dict(torch.load('mlp_weights.pth'))
model.eval()

criterion = nn.MSELoss()
test_loss = 0
with torch.no_grad():
    for inputs, targets in test_loader:
        targets = targets.view(inputs.size(0), -1)
        outputs = model(inputs)
        test_loss += criterion(outputs, targets).item()
test_loss /= len(test_loader)

print(f"Final Test Loss = {test_loss:.4f}")


100%|███████████████████████████████████████| 1000/1000 [00:15<00:00, 66.21it/s]


Epoch 1, Train Loss = 0.0198, Val Loss = 0.0143
Epoch 2, Train Loss = 0.0140, Val Loss = 0.0139
Epoch 3, Train Loss = 0.0136, Val Loss = 0.0136
Epoch 4, Train Loss = 0.0121, Val Loss = 0.0106
Epoch 5, Train Loss = 0.0100, Val Loss = 0.0095
Epoch 6, Train Loss = 0.0089, Val Loss = 0.0083
Epoch 7, Train Loss = 0.0081, Val Loss = 0.0078
Epoch 8, Train Loss = 0.0074, Val Loss = 0.0068
Epoch 9, Train Loss = 0.0066, Val Loss = 0.0064
Epoch 10, Train Loss = 0.0061, Val Loss = 0.0057
Epoch 11, Train Loss = 0.0055, Val Loss = 0.0052
Epoch 12, Train Loss = 0.0051, Val Loss = 0.0057
Epoch 13, Train Loss = 0.0048, Val Loss = 0.0046
Epoch 14, Train Loss = 0.0044, Val Loss = 0.0041
Epoch 15, Train Loss = 0.0041, Val Loss = 0.0041
Epoch 16, Train Loss = 0.0040, Val Loss = 0.0041
Epoch 17, Train Loss = 0.0037, Val Loss = 0.0038
Epoch 18, Train Loss = 0.0036, Val Loss = 0.0033
Epoch 19, Train Loss = 0.0032, Val Loss = 0.0031
Epoch 20, Train Loss = 0.0030, Val Loss = 0.0029
Epoch 21, Train Loss = 0.0029

## Prédictions

In [58]:
main_inference(4,8,3)

Image sauvegardée dans le fichier predictions


## Analyse en composantes principales 

In [ ]:
def apply_pca(X, n_components):
    pca = PCA(n_components=n_components)
    
    # Projection
    X_pca = pca.fit_transform(X)
    
    # Reconstruction
    X_reconstructed = pca.inverse_transform(X_pca)
    
    return pca, X_pca, X_reconstructed